<a href="https://colab.research.google.com/github/29bharat/Big_Data/blob/master/PySpark_Challenge16.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
# Install Java, Spark, and Findspark
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://www-us.apache.org/dist/spark/spark-2.4.5/spark-2.4.5-bin-hadoop2.7.tgz
!tar xf spark-2.4.5-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-2.4.5-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

In [0]:
!wget https://jdbc.postgresql.org/download/postgresql-42.2.9.jar

--2020-05-03 22:54:39--  https://jdbc.postgresql.org/download/postgresql-42.2.9.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 914037 (893K) [application/java-archive]
Saving to: ‘postgresql-42.2.9.jar.2’

postgresql-42.2.9.j 100%[===================>] 892.61K  4.75MB/s    in 0.2s    

2020-05-03 22:54:40 (4.75 MB/s) - ‘postgresql-42.2.9.jar.2’ saved [914037/914037]



In [0]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("CloudETL").config("spark.driver.extraClassPath","/content/postgresql-42.2.9.jar").getOrCreate()

In [0]:
# Read in data from S3 Buckets
from pyspark import SparkFiles
url ="https://pyspark-challenge16.s3-us-west-1.amazonaws.com/amazon_reviews_us_Shoes_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
giftcard_df = spark.read.csv(SparkFiles.get("amazon_reviews_us_Shoes_v1_00.tsv.gz"), sep="\t", header=True, inferSchema=True)

# Show DataFrame
giftcard_df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-------------------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|        review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-------------------+
|         US|   18069663|R3P2HIOQCIN5ZU|B000XB31C0|     265024781|Minnetonka Men's ...|           Shoes|          1|            0|          0|   N|                Y|                   .|Do not buy: reall...|2015-08-31 00:00:00|
|         US|   16251825|R12VVR0WH5Q24V|B00CFYZH5W|     259035853|Teva Men's Pajaro...| 

In [0]:
# To get the number of rows in the df
giftcard_df.count()

4366916

In [0]:
# Drop NULL values
giftcard_df = giftcard_df.dropna()
giftcard_df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-------------------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|        review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-------------------+
|         US|   18069663|R3P2HIOQCIN5ZU|B000XB31C0|     265024781|Minnetonka Men's ...|           Shoes|          1|            0|          0|   N|                Y|                   .|Do not buy: reall...|2015-08-31 00:00:00|
|         US|   16251825|R12VVR0WH5Q24V|B00CFYZH5W|     259035853|Teva Men's Pajaro...| 

In [0]:
giftcard_df.count()

4366324

In [0]:
giftcard_df.dtypes

[('marketplace', 'string'),
 ('customer_id', 'int'),
 ('review_id', 'string'),
 ('product_id', 'string'),
 ('product_parent', 'int'),
 ('product_title', 'string'),
 ('product_category', 'string'),
 ('star_rating', 'int'),
 ('helpful_votes', 'int'),
 ('total_votes', 'int'),
 ('vine', 'string'),
 ('verified_purchase', 'string'),
 ('review_headline', 'string'),
 ('review_body', 'string'),
 ('review_date', 'timestamp')]

In [0]:

# Load the functions to convert to date
from pyspark.sql.functions  import date_format
from pyspark.sql.types import DateType

# Updated df with a new column with format yyyy-mm-dd
giftcard_df = giftcard_df.withColumn("new_review_date", giftcard_df['review_date'].cast(DateType()))
giftcard_df = giftcard_df.drop("review_date")
giftcard_df = giftcard_df.withColumnRenamed("new_review_date","review_date")
giftcard_df.show()



+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   18069663|R3P2HIOQCIN5ZU|B000XB31C0|     265024781|Minnetonka Men's ...|           Shoes|          1|            0|          0|   N|                Y|                   .|Do not buy: reall...| 2015-08-31|
|         US|   16251825|R12VVR0WH5Q24V|B00CFYZH5W|     259035853|Teva Men's Pajaro...|           Shoes|          5|    

In [0]:
giftcard_df.count()

4366324

In [0]:
giftcard_df.dtypes

[('marketplace', 'string'),
 ('customer_id', 'int'),
 ('review_id', 'string'),
 ('product_id', 'string'),
 ('product_parent', 'int'),
 ('product_title', 'string'),
 ('product_category', 'string'),
 ('star_rating', 'int'),
 ('helpful_votes', 'int'),
 ('total_votes', 'int'),
 ('vine', 'string'),
 ('verified_purchase', 'string'),
 ('review_headline', 'string'),
 ('review_body', 'string'),
 ('review_date', 'date')]

In [0]:
# Create user dataframe to match review_id table created on pgadmin
review_id_table = giftcard_df.select(["review_id", "customer_id", "product_id", "product_parent", "review_date"])
review_id_table.show()

+--------------+-----------+----------+--------------+-----------+
|     review_id|customer_id|product_id|product_parent|review_date|
+--------------+-----------+----------+--------------+-----------+
|R3P2HIOQCIN5ZU|   18069663|B000XB31C0|     265024781| 2015-08-31|
|R12VVR0WH5Q24V|   16251825|B00CFYZH5W|     259035853| 2015-08-31|
| RNCCKB6TV5EEF|   20381037|B00S8JNN3Q|     666066660| 2015-08-31|
|R2NZXYIVCGB13W|     108364|B00XFBPOQG|     448483263| 2015-08-31|
|R2EQ1TG9IT3OEQ|   45449350|B00SW64Y9W|       7853171| 2015-08-31|
|R1WXA9JSC2H1U1|   19324665|B011F9E6LI|      14311457| 2015-08-31|
|R12ENYLFGGNWRV|   50073594|B00HAUP1OI|     264821602| 2015-08-31|
|R2R07E5PNXEUO5|   21706057|B00L1RKOKW|     767118055| 2015-08-31|
|R27BA52AKWMWN3|   13708216|B005WA9MSE|     813856438| 2015-08-31|
| RLF8DOID2KD5O|   40542649|B00BEYQI5C|     661491213| 2015-08-31|
|R369CEXHXHC6NQ|   13409446|B00EYAFTCQ|     332158165| 2015-08-31|
|R171PPIJXFONVI|    9451727|B00I0QHY32|      49243908| 2015-08

In [0]:
review_id_table.dtypes

[('review_id', 'string'),
 ('customer_id', 'int'),
 ('product_id', 'string'),
 ('product_parent', 'int'),
 ('review_date', 'date')]

In [0]:
# Create user dataframe to match products table created on pgadmin
products_table = giftcard_df.select(["product_id", "product_title"])
products_table.show()

+----------+--------------------+
|product_id|       product_title|
+----------+--------------------+
|B000XB31C0|Minnetonka Men's ...|
|B00CFYZH5W|Teva Men's Pajaro...|
|B00S8JNN3Q|Anne Klein Perfec...|
|B00XFBPOQG|adidas Men's 10K ...|
|B00SW64Y9W|OverBling Sneaker...|
|B011F9E6LI|MESSI 15.3 FG/AG ...|
|B00HAUP1OI|Hoka One One Men'...|
|B00L1RKOKW|Olukai Nohea Mesh...|
|B005WA9MSE|Carolina Mens 6" ...|
|B00BEYQI5C|Alegria Women's S...|
|B00EYAFTCQ|Naturalizer Women...|
|B00I0QHY32|Forever Link Wome...|
|B010FZZKYA|Versace Collectio...|
|B00V8B30K2|Twisted Girl's Ch...|
|B00LAVB1TC|Travel Smart Hand...|
|B003C1P8B0|Saucony Originals...|
|B000W3UL7W|Dr. Marten's Wome...|
|B00LX65PQO|Skechers Performa...|
|B00BEE7N3I|Hi-Tec Kid's Nepa...|
|B008NCHMBW|Foot Sox Original...|
+----------+--------------------+
only showing top 20 rows



In [0]:
products_table.dtypes

[('product_id', 'string'), ('product_title', 'string')]

In [0]:
# Count before removing duplicates
products_table.count()

4366324

In [0]:
# Drop duplicates in product table and count of rows
products_table = products_table.dropDuplicates()
products_table.count()

1900895

In [0]:
products_table.show()

+----------+--------------------+
|product_id|       product_title|
+----------+--------------------+
|B001CJL5ES|L.B. Evans Men's ...|
|B002MCVHUQ|New Balance Insol...|
|B00UXBFW6S|Kenox Sling Backp...|
|B002CMM4M6|Capezio Women's 1...|
|B00HNO32BU|Timberland PRO Me...|
|B00C6BQQW2|New Balance Men's...|
|B00M0NREE0|New Balance Women...|
|B00ZC4VIKK|Donalworld Women ...|
|B00291CGH8|Justin Boots Wome...|
|B000P6GKOO|Nike Men's Rosher...|
|B00IEB8BQ2|Khan New 2014 Uni...|
|B00IBGJKW4|Powerstep Ultrast...|
|B008KK1D5S|Clarks Women's Br...|
|B003IHV9XE|Birkenstock Women...|
|B00M9500FY|FitFlop Women's L...|
|B00R59ZBXU|Keds Women's Doub...|
|B00KQ66NBS|New Balance Men's...|
|B00E41QYRE|Jambu Women's Xte...|
|B0106QF79Q|Sanita Women's Pr...|
|B0084ZF7V0|DV by Dolce Vita ...|
+----------+--------------------+
only showing top 20 rows



In [0]:
# Create user dataframe to match customers table created on pgadmin and rename the aggregate column to match the column name
customers_table = giftcard_df.groupBy("customer_id").count().withColumnRenamed("count", "customer_count")
customers_table.dtypes


[('customer_id', 'int'), ('customer_count', 'bigint')]

In [0]:
customers_table.show()

+-----------+--------------+
|customer_id|customer_count|
+-----------+--------------+
|   16121565|             5|
|   48146680|             1|
|   11062912|             1|
|   51451778|             1|
|   27920838|             1|
|    4919528|             3|
|   47802851|             2|
|   42560427|             1|
|   29467780|             1|
|   49703087|            12|
|   38456994|             4|
|     557836|             1|
|   43783459|             1|
|   23607725|             1|
|   45632184|             1|
|   19979069|             2|
|   28380959|             1|
|   46563381|             1|
|   21628739|             3|
|   23042837|             2|
+-----------+--------------+
only showing top 20 rows



In [0]:
# Load in sql functions to use columns
from pyspark.sql.functions import col

# Create user dataframe to match vine table created on pgadmin
vine_table = giftcard_df.filter(col("vine")== "Y")
vine_table = vine_table.select(["review_id", "star_rating", "helpful_votes", "total_votes", "vine"])
vine_table.show()

+--------------+-----------+-------------+-----------+----+
|     review_id|star_rating|helpful_votes|total_votes|vine|
+--------------+-----------+-------------+-----------+----+
| RISSSMBIV0J9B|          5|            5|          6|   Y|
|R3J67UUFB5N1JM|          5|            0|          0|   Y|
|R17XQZGLB6UDER|          4|            0|          0|   Y|
|R391PM81J6G5JH|          5|            1|          1|   Y|
| RSFQY2YR8A3AC|          4|            3|          5|   Y|
| RLFBSXQA2JUHR|          5|            0|          0|   Y|
|R13P2OICQ0MTLM|          4|            0|          1|   Y|
|R1P9EMSVXTU8QQ|          5|            0|          0|   Y|
|R3BQSPJ27GKZDK|          5|           10|         14|   Y|
| RAFKWU1V19FXT|          5|            4|          4|   Y|
|R1XB4YP0M7YQXD|          5|            1|          1|   Y|
|R1KGCJI9MAXQRC|          5|            0|          0|   Y|
|R2GP7RXE3BFJI9|          5|            1|          1|   Y|
| RRI85XFSGHF36|          5|            

In [0]:
vine_table.dtypes

[('review_id', 'string'),
 ('star_rating', 'int'),
 ('helpful_votes', 'int'),
 ('total_votes', 'int'),
 ('vine', 'string')]

In [0]:
vine_table.count()

895

In [0]:
# Load in sql functions to use columns
from pyspark.sql.functions import col

# Create user dataframe to match non-vine table created on pgadmin
non_vine_table = giftcard_df.filter(col("vine")== "N")
non_vine_table = non_vine_table.select(["review_id", "star_rating", "helpful_votes", "total_votes", "vine"])
non_vine_table.show()

+--------------+-----------+-------------+-----------+----+
|     review_id|star_rating|helpful_votes|total_votes|vine|
+--------------+-----------+-------------+-----------+----+
|R3P2HIOQCIN5ZU|          1|            0|          0|   N|
|R12VVR0WH5Q24V|          5|            0|          0|   N|
| RNCCKB6TV5EEF|          4|            0|          0|   N|
|R2NZXYIVCGB13W|          5|            0|          6|   N|
|R2EQ1TG9IT3OEQ|          3|            0|          0|   N|
|R1WXA9JSC2H1U1|          5|            1|          1|   N|
|R12ENYLFGGNWRV|          5|            1|          1|   N|
|R2R07E5PNXEUO5|          4|            0|          0|   N|
|R27BA52AKWMWN3|          5|            0|          0|   N|
| RLF8DOID2KD5O|          3|            0|          0|   N|
|R369CEXHXHC6NQ|          5|            0|          0|   N|
|R171PPIJXFONVI|          5|            0|          0|   N|
|R2JDNM8F2P06FU|          4|            1|          1|   N|
|R2W977FO4M97XT|          5|            

In [0]:
# Configure settings for RDS
mode = "append"
jdbc_url="jdbc:postgresql://dataviz.c8suxpm4okhy.us-east-2.rds.amazonaws.com:5432/PySpark_Challenge16"
config = {"user":"postgres", 
          "password": "Ready2go", 
          "driver":"org.postgresql.Driver"}

In [0]:
# Write DataFrame to review_id_table table in RDS
review_id_table.write.jdbc(url=jdbc_url, table='review_id_table', mode=mode, properties=config)

In [0]:
# Write dataframe to billing_info table in RDS
customers_table.write.jdbc(url=jdbc_url, table='customers', mode=mode, properties=config)

In [0]:
# Write dataframe to payment_info table in RDS
products_table.write.jdbc(url=jdbc_url, table='products', mode=mode, properties=config)

In [0]:
# Write dataframe to payment_info table in RDS
vine_table.write.jdbc(url=jdbc_url, table='vine_table', mode=mode, properties=config)

In [0]:
# Write dataframe to payment_info table in RDS
non_vine_table.write.jdbc(url=jdbc_url, table='non_vine_table', mode=mode, properties=config)